In [1]:
import pandas as pd

In [2]:
#api활용을 위해 제공되고 있는 xycode에서 부산 code만 추출한다

In [3]:
data = pd.read_excel('../Data/rawdata/API_xycode.xlsx')

In [4]:
data['CODE'] = data['행정구역코드'].apply(str)

In [5]:
busan_data = data[data['CODE'].str.startswith('26')]

In [6]:
busan_data = busan_data[['2단계','3단계','격자 X', '격자 Y', 'CODE']]

In [7]:
busan_data = busan_data.fillna(0)
busan_data = busan_data[busan_data['3단계']==0]

In [8]:
busan_data = busan_data.drop(columns='3단계')
busan_data = busan_data.drop(index=451)
busan_data

,2단계,격자 X,격자 Y,CODE
452,중구,97,74,2611000000
462,서구,97,74,2614000000
476,동구,98,75,2617000000
489,영도구,98,74,2620000000
501,부산진구,97,75,2623000000
522,동래구,98,76,2626000000
536,남구,98,75,2629000000
554,북구,96,76,2632000000
568,해운대구,99,75,2635000000
587,사하구,96,74,2638000000


In [9]:
busan_data = busan_data.rename(columns={'2단계':'ZONE','격자 X':'X','격자 Y' : 'Y'})

In [10]:
busan = busan_data.reset_index(drop=True)

In [11]:
busan

,ZONE,X,Y,CODE
0,중구,97,74,2611000000
1,서구,97,74,2614000000
2,동구,98,75,2617000000
3,영도구,98,74,2620000000
4,부산진구,97,75,2623000000
5,동래구,98,76,2626000000
6,남구,98,75,2629000000
7,북구,96,76,2632000000
8,해운대구,99,75,2635000000
9,사하구,96,74,2638000000


In [12]:
busan_data.to_csv('../Data/final_data/DB/xycode.csv', encoding='utf-8', index=False)

In [13]:
# busan_dong을 위한 코드를 만든다.

In [14]:
busan_dong = pd.read_csv('../Data/processing_data/1st/busan_flood_geo.csv')

busan_dong = busan_dong[['ADM_CD','ZONE','DONG','SLOPE_AVG','HIGH','FLUD_DPWT.1']]

busan_dong = busan_dong.sort_values('FLUD_DPWT.1', ascending=False,ignore_index=True)

busan_dong = busan_dong.drop_duplicates(['DONG'], keep='first')\
.sort_values(['ZONE','DONG'], ascending=['FALSE','TRUE'])\
.reset_index(drop=True)

busan_dong = busan_dong.drop(columns='FLUD_DPWT.1')

new_data={'ADM_CD':2617053000,'ZONE':'동구','DONG':'초량동','SLOPE_AVG':'1','HIGH':'4'}
busan_dong = busan_dong.append(new_data, ignore_index=True)

busan_dong = busan_dong[['ADM_CD','ZONE','SLOPE_AVG','HIGH']]
busan_dong['SLOPE_AVG'] = busan_dong['SLOPE_AVG'].astype(float)
busan_dong['HIGH'] = busan_dong['HIGH'].astype(float)

In [23]:
slope_weight = busan_dong['SLOPE_AVG'].copy()
high_weight = busan_dong['HIGH'].copy()

#경사도가중치
for idx,col in enumerate(busan_dong['SLOPE_AVG']):
    if col <= 10:
        slope_weight[idx] = col / (41.67 / 5**2 )
    elif col > 10 and col <= 15:
        slope_weight[idx] = col / (25 / 4**2)
    elif col > 15 and col <= 25:
        slope_weight[idx] = col / (16.67 / 3**2)
    elif col > 25 and col <= 50:
        slope_weight[idx] = col / (8.33 / 2**2)
    elif col > 50:
        slope_weight[idx] = col / (8.33 / 1**2)
#고도 가중치
for idx,col in enumerate(busan_dong['HIGH']):
    if col <= 20:
        high_weight[idx] = col / (79.62 / 1**2)
    elif col > 20 and col <= 40:
        high_weight[idx] = col / (10.18 / 2**2)
    elif col > 40 and col <= 60:
        high_weight[idx] = col / (4.63 / 3**2)
    elif col > 60 and col <= 80:
        high_weight[idx] = col / (2.78 / 4**2)
    elif col > 80:
        high_weight[idx] = col / (2.78 / 5**2)
busan_dong['SLOPE_AVG_Weight'] = slope_weight
busan_dong['HIGH_Weight'] = high_weight
busan_dong = busan_dong.drop(columns=['SLOPE_AVG','HIGH'])

In [29]:
busan_dong

,ADM_CD,ZONE,SLOPE_AVG_Weight,HIGH_Weight
0,2644012000,강서구,12.147570,0.114418
1,2644000000,강서구,0.599952,0.022231
2,2644000000,강서구,0.599952,0.014820
3,2644010400,강서구,0.599952,0.019467
4,2644011900,강서구,7.040000,0.016453
5,2641000000,금정구,7.040000,9.803536
6,2641000000,금정구,2.699784,0.086410
7,2641000000,금정구,2.699784,14.231827
8,2641000000,금정구,7.040000,367.021583
9,2641000000,금정구,0.599952,356.661871


In [30]:
sig_info = pd.read_csv('../Data/processing_data/2nd/sig_info_weight.csv')

In [32]:
busan_dong = pd.merge(busan_dong, sig_info, on='ZONE', how='left')

In [33]:
busan_dong.to_csv('../Data/final_data/DB/busan_base_data.csv', encoding='utf-8', index=False)

In [34]:
########REALdata 학습한 데이터 만들기